In [4]:
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,confusion_matrix,roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

np.random.seed(0);
X = np.genfromtxt("data/X_train.txt",delimiter = None)
Y = np.genfromtxt("data/Y_train.txt",delimiter = None)
test_data = np.genfromtxt("data/X_test.txt",delimiter = None)
# feature extraction
model = LogisticRegression(class_weight = 'balanced')
rfe = RFE(model, 5)
fit = rfe.fit(X, Y)
print(fit.n_features_)
print(fit.support_)
print(fit.ranking_)

5
[ True False False False False False False False  True  True  True  True
 False False]
[ 1  6  5  3 10  9  8  2  1  1  1  1  4  7]


In [5]:
nn_predictions = np.zeros(shape=(2,1))
Ytest = np.array([1,2])

np.sum(np.equal(nn_predictions,Ytest))

0

In [6]:
sc = StandardScaler()

X = X[:, [0,8,9,10,11]]
sc.fit(test_data)
test_data = sc.transform(test_data)

In [9]:
from imblearn.combine import SMOTEENN
from sklearn.externals import joblib

Y_sub = np.zeros(shape=(1,200000))
smoteen = SMOTEENN(random_state=0)
Xtest,Ytest = smoteen.fit_sample(X,Y)
sc2 = StandardScaler()
sc2.fit(Xtest)
Xtest = sc2.transform(Xtest)
j = 1
nn_predictions = np.zeros(shape=(Xtest.shape[0],51))

for i in range(0,51):
    model = MLPClassifier(activation='relu',solver='lbfgs',hidden_layer_sizes=(300+2*i,300+2*i),early_stopping=True,batch_size=1)
    X_train, _, y_train, _ = train_test_split(X, Y, test_size=0.25, random_state=j, stratify=Y, shuffle = True)
    sc1 = StandardScaler()
    sc1.fit(X_train)
    X_train = sc1.transform(X_train)
    smoteen1 = SMOTEENN(random_state=j)
    x_smoteenn1,y_smoteenn1 = smoteen1.fit_sample(X_train,y_train)
    model.fit(x_smoteenn,y_smoteenn)
    file_name = 'model_'+str(i)+'.pkl'
    joblib.dump(model,file_name)
    nn_predictions[:,i] = model.predict(Xtest)
    np.savetxt("data/predictions.txt",nn_predictions)
    j += 1

KeyboardInterrupt: 

In [ ]:
weight_vector = np.zeros(shape=(51,1))

for i in range(0,51):
    weight_vector[i] = np.sum(np.equal(nn_predictions[:,i],Ytest))
    
sum_vector = float(np.sum(weight_vector))
final_weights = weight_vector/sum_vector
final_weights

In [ ]:
np.savetxt("data/final_weights_voting_ensemble.txt",final_weights)
nn_predictions = np.zeros(shape=(200000,51))

for i in range(0,51):
    file_name = 'model_'+str(i)+'.pkl'
    model = joblib.load(file_name)
    nn_predictions[:,i] = model.predict_proba(test_data[:,[0,8,9,10,11]])[:,1]
    np.savetxt("data/prelim_predictions.txt",nn_predictions)

Y_sub = nn_predictions * final_weights

Y_sub = np.vstack([np.arange(test_data.shape[0]), Y_sub]).T

np.savetxt("data/Y_sub.txt", Y_sub, '%d, %.5f', header='ID,Prob1', comments='', delimiter=',')